# GenAI powered skill explorer for students and professionals

## Introduction
In this notebook we experiment with the use of LLMs for helping students and professionals seeking to improve skills for a specific job role. SkillSensei is meant to help by:

- **Matching job roles with trainable skills:** Identify appropriate roles and career avenues based on personal skills and professional skills.

## Table of Contents:
This notebook is separated into the follwing sections

- **Section 1: Setup**
- **Section 2: Initial experimentation**
- **Section 3: Introducing RAG**
- **Section 4: Evaluation**

## Concepts from the course:
In this notebook, we explore the following concepts from the LLM Foundations course:

- Structured output/JSON mode/controlled generation
- Few-shot prompting 
- Gen AI evaluation
- Embeddings
- Retrieval augmented generation (RAG)
- Vector search/vector store/vector database

# Section 1: Setup

Install the SDK

In [2]:
# !pip uninstall -qqy jupyterlab  # Remove unused conflicting packages
# !pip install -qU "google-genai==1.7.0" "chromadb==0.6.3"

Import the SDK and some helpers for rendering the output.

In [7]:
from google import genai
from google.genai import types
import os
from dotenv import load_dotenv

In [4]:
from IPython.display import HTML, Markdown, display

Set up a retry helper. This allows you to "Run all" without worrying about per-minute quota.

In [6]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

## Set up your API key
To run the following cell, your API key must be stored it in a Kaggle secret named **GOOGLE_API_KEY**.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/apikey).

To make the key available through Kaggle secrets, choose Secrets from the Add-ons menu and follow the instructions to add your key or enable it for this notebook.

In [8]:
load_dotenv()

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')


# from kaggle_secrets import UserSecretsClient

# GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# Section 2: Initial exploration
Let's begin by exploring the Gemini's capabilities to provide career advice, by trying out a few simple prompts, and experimenting with Chain of Though and ReAct. 

In [9]:
from pprint import pprint

client = genai.Client(api_key=GOOGLE_API_KEY)

# for model in client.models.list():
#     pprint(f"{model.display_name}: {model.description}")

## Prompting techniques

In [25]:
model_config = types.GenerateContentConfig(
    # These are the default values for gemini-2.0-flash.
    temperature=1.0,
    top_p=0.95,
)

zero_shot_prompt = """You are an career advisor focusing on skills. Provide a list of the most relevant skills for each of the following jobs: data engineer, school teacher, barber, and software engineer.:
Most relevant skills: """

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=zero_shot_prompt)

Markdown(response.text)

Okay, here's a breakdown of the most relevant skills for each of the listed jobs, keeping in mind that specific requirements can vary based on industry, location, and seniority:

**1. Data Engineer**

*   **Technical/Hard Skills:**

    *   **Data Modeling:** Designing efficient and scalable data structures (schemas, tables, etc.)
    *   **ETL (Extract, Transform, Load) Processes:** Building and maintaining pipelines to move and transform data from various sources.
    *   **Data Warehousing:** Experience with data warehouse solutions (e.g., Snowflake, Redshift, BigQuery).
    *   **Database Management Systems (DBMS):** Proficiency in SQL and NoSQL databases (e.g., MySQL, PostgreSQL, MongoDB, Cassandra).
    *   **Cloud Computing:** Expertise with cloud platforms (e.g., AWS, Azure, GCP) and their data-related services.
    *   **Programming Languages:** Strong skills in Python and/or Java/Scala.
    *   **Big Data Technologies:** Familiarity with Hadoop, Spark, Kafka, etc., for processing large datasets.
    *   **Data Governance:** Understanding and implementing data quality, security, and compliance measures.
    *   **Operating Systems:** Knowledge of Linux and/or Unix based platforms.
    *   **Version Control:** Git and other version control systems.
*   **Soft Skills:**

    *   **Problem-Solving:** Ability to identify and resolve data-related issues effectively.
    *   **Analytical Thinking:** Analyzing data requirements and designing appropriate solutions.
    *   **Communication:** Clearly explaining technical concepts to both technical and non-technical audiences.
    *   **Collaboration:** Working effectively with data scientists, analysts, and other stakeholders.
    *   **Attention to Detail:** Ensuring data accuracy and integrity.
    *   **Adaptability:** Keeping up-to-date with evolving technologies and data trends.
    *   **Time Management:** Ability to manage work load to effectively meet deadlines.

**2. School Teacher**

*   **Technical/Hard Skills:**

    *   **Curriculum Development:** Creating and implementing engaging and effective lesson plans.
    *   **Subject Matter Expertise:** Strong knowledge of the subject(s) being taught.
    *   **Classroom Management:** Creating a positive and productive learning environment.
    *   **Assessment:** Designing and administering assessments (tests, quizzes, projects) to evaluate student learning.
    *   **Educational Technology:** Using technology tools and platforms to enhance teaching and learning (e.g., learning management systems, educational apps).
    *   **Differentiated Instruction:** Adapting teaching methods to meet the diverse needs of students.
    *   **Reporting:** Effectively summarizing student progress to present to parents/guardians.
*   **Soft Skills:**

    *   **Communication:** Communicating effectively with students, parents, and colleagues.
    *   **Patience:** Maintaining a calm and supportive demeanor, especially when dealing with challenging students.
    *   **Empathy:** Understanding and responding to students' emotional needs.
    *   **Creativity:** Developing innovative and engaging lessons and activities.
    *   **Organization:** Managing classroom materials, schedules, and student records.
    *   **Leadership:** Guiding and motivating students to achieve their learning goals.
    *   **Adaptability:** Adjusting teaching strategies based on student progress and classroom dynamics.
    *   **Problem Solving:** Effectively deal with any classroom issues.

**3. Barber**

*   **Technical/Hard Skills:**

    *   **Haircutting:** Proficiency in various cutting techniques (e.g., fades, tapers, layers).
    *   **Shaving:** Expertise in shaving techniques, including straight razor shaving (where permitted).
    *   **Styling:** Ability to style hair using different products and techniques.
    *   **Beard Trimming & Shaping:** Skill in trimming and shaping beards to complement facial features.
    *   **Hair Coloring:** Knowledge of hair coloring techniques and products.
    *   **Sanitation & Hygiene:** Following strict hygiene protocols to maintain a clean and safe work environment.
    *   **Tool Maintenance:** Properly cleaning, sanitizing, and maintaining barbering tools.
*   **Soft Skills:**

    *   **Customer Service:** Providing excellent service and building rapport with clients.
    *   **Communication:** Listening to clients' needs and providing clear explanations of services.
    *   **Attention to Detail:** Ensuring precision and accuracy in haircuts and other services.
    *   **Creativity:** Suggesting styles and looks that suit clients' individual features.
    *   **Interpersonal Skills:** Building relationships with clients and creating a welcoming atmosphere.
    *   **Time Management:** Managing appointments and completing services efficiently.
    *   **Physical Stamina:** Standing for extended periods and performing repetitive motions.
    *   **Sales:** Promoting and recommending products and services to clients.

**4. Software Engineer**

*   **Technical/Hard Skills:**

    *   **Programming Languages:** Proficiency in one or more languages (e.g., Python, Java, C++, JavaScript, C#).
    *   **Data Structures and Algorithms:** Understanding fundamental data structures (e.g., arrays, linked lists, trees) and algorithms (e.g., sorting, searching).
    *   **Software Design Principles:** Knowledge of design patterns, SOLID principles, and other software design concepts.
    *   **Version Control:** Using Git and other version control systems for collaborative development.
    *   **Testing:** Writing unit tests, integration tests, and other types of tests to ensure code quality.
    *   **Debugging:** Identifying and resolving software bugs efficiently.
    *   **Databases:** Working with relational and/or NoSQL databases.
    *   **Operating Systems:** Understanding operating system concepts and working with different OS environments (e.g., Linux, Windows, macOS).
    *   **Web Development:** (If applicable) Knowledge of front-end (HTML, CSS, JavaScript) and/or back-end technologies (e.g., Node.js, Python frameworks like Django or Flask, Java frameworks like Spring).
    *   **Cloud Computing:** (Increasingly important) Familiarity with cloud platforms and services.
*   **Soft Skills:**

    *   **Problem-Solving:** Breaking down complex problems into smaller, manageable tasks.
    *   **Analytical Thinking:** Analyzing requirements and designing effective solutions.
    *   **Communication:** Clearly communicating technical concepts to both technical and non-technical audiences.
    *   **Collaboration:** Working effectively in a team environment.
    *   **Time Management:** Meeting deadlines and managing workload effectively.
    *   **Adaptability:** Learning new technologies and adapting to changing project requirements.
    *   **Attention to Detail:** Writing clean, well-documented, and error-free code.
    *   **Continuous Learning:** Staying up-to-date with the latest technologies and trends.

Remember that this is not an exhaustive list, and the specific skills required for each job may vary depending on the specific role and employer. However, these are some of the most important and relevant skills to focus on. Good luck!


While this list is already quite extensive, there are a few things to note. First, while the zero shot prompt only mentioned "most relevant" skills, the LLM generated more than just skills. For example, it also generated qualities such as "strength and lifting capacity" which while are trainable, can be thought of as physical qualities rather than skills. Second, while it provided a structured answer (i.e., grouped skills and qualities in groups) it generated an answer which can be too broad and overwhelming for a user. Instead we want, {NAME} to provide more focused skills to users.  

In [26]:
import typing_extensions as typing
import json

class JobSkills(typing.TypedDict):
    skills: list[str]
    descriptions: list[str]

few_shot_prompt = """Based on a job title provide a list of skills and descriptions in valid JSON :

EXAMPLE:
astronaut
JSON Response:
```
{
skills: ["STEM Proficiency", "Leadership", "Giving First Aid", "Data Analysis & Interpretation"]
descriptions: ["Strong foundation in science, technology, engineering, and mathematics (physics, chemistry, biology, astronomy, computer science, engineering principles)", "Taking initiative, making sound judgments, and leading effectively in critical situations.", "Basic medical training to handle emergencies and provide first aid in isolated environments", "Analyzing data from experiments, observations, and equipment monitoring"]
}
```

EXAMPLE:
school teacher
JSON Response:
```
{
"skills": ["Presentation Skills", "Curriculum Development", "Deep Content Knowledge", "Assessment & Evaluation", "Classroom Managment"]
"descriptions": ["Ability to present information clearly, engagingly, and effectively.", "Ability to create engaging and effective lesson plans and educational materials.", "A thorough understanding of the subject(s) being taught.", "Ability to design and administer assessments, analyze student performance, and provide constructive feedback.", "Planning lessons, managing resources, and tracking student progress efficiently.",]
}
```

systems administrator
"""

baseline_response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=JobSkills,
    ),
    contents=few_shot_prompt)

data = json.loads(baseline_response.text)

for i in range(len(data.get("skills"))):
    print(data.get("skills")[i], ": ", data.get("descriptions")[i])

Operating Systems :  Proficiency in managing and maintaining various operating systems (Windows, Linux, macOS).
Network Administration :  Configuring and maintaining network infrastructure, including routers, switches, and firewalls.
Scripting :  Ability to write scripts in languages like Python, Bash, or PowerShell to automate tasks.
Troubleshooting :  Identifying and resolving hardware, software, and network issues.
Automation :  Automating repetitive tasks using tools and scripts to improve efficiency and reduce errors


This is an improvement over the zero shot prompting, as it provides a short list of skills a person can focus on. The use of json for the output, also it more structured and easily parsable by other tools. One thing to notice is however, that the model could have learned to limit the output to four skills per prompt, because both examples include four skills. While the focused scope is not necessarily a disadvantage, we don't know how the model prioritizes the selected skills.

# Section 3: Introducing RAG

Now let's explore how the listed skills compare, once we use a vectorized dataset containing job listings and their relevant skills scrapped from Linkedin during 2024. 

## The dataset
As the entire dataset contains 1.3M Jobs and Skills we will use just a subset of the data. The full dataset is available on Kaggle: [1.3M Linkedin Jobs & Skills (2024)](https://www.kaggle.com/datasets/asaniczka/1-3m-linkedin-jobs-and-skills-2024?select=linkedin_job_postings.csv)

In [43]:
import pandas as pd

# Load the CSVs
job_postings_df = pd.read_csv("./archive/linkedin_job_postings.csv", nrows=20000)
skills_df = pd.read_csv("./archive/job_skills.csv", nrows=20000)

job_postings_small_df = job_postings_df[["job_link", "job_title"]]

# Optional: clean nulls
skills_df.dropna(inplace=True)
#summary_df.dropna(inplace=True)

In [44]:
# Merge on job_link
merged_df = pd.merge(job_postings_small_df, skills_df, on='job_link', how='inner')

#merged_df = pd.merge(merged_df, skills_df, on='job_link', how='inner')

merged_df.head(7)

,job_link,job_title,job_skills
0,https://www.linkedin.com/jobs/view/systems-adm...,Systems Administrator,"Windows computing, Server 2012, Software troub..."
1,https://www.linkedin.com/jobs/view/enrollment-...,Enrollment Specialist Team Lead,"Customer service, Goal achievement, Teamwork, ..."
2,https://www.linkedin.com/jobs/view/senior-risk...,"Senior Risk Manager, Interest Rate Risk in the...","Interest Rate Risk Management, Asset Liability..."
3,https://www.linkedin.com/jobs/view/prosthodont...,Prosthodontist/Periodontist,"Prosthodontics, Periodontics, Stateoftheart di..."
4,https://www.linkedin.com/jobs/view/veterinary-...,Veterinary Receptionist,"Customer service, Organizational skills, Commu..."
5,https://www.linkedin.com/jobs/view/senior-stru...,Senior Structures Engineer,"CSiBridge, Conspan, RC Pier, BRASS, MDX, Highw..."
6,https://www.linkedin.com/jobs/view/field-techn...,Field Technician,"Communication skills, Ability to work at heigh..."


In [45]:
# Combine job_summary and job_skills into one field
merged_df['text_to_embed'] = (
    "Title: " + merged_df['job_title'].astype(str).str.strip() + "\n" +
    "Skills: " + merged_df['job_skills'].astype(str).str.strip()
)

documents = merged_df['text_to_embed'].tolist()

In [33]:
documents

["Title: Systems Administrator\nSkills: Windows computing, Server 2012, Software troubleshooting, Analytical thinking, Problem solving, PowerShell, Customer service, Microsoft Office 365, Azure AD, SharePoint, Security and Compliance Center, Teams, MCSE, MCSA, Bachelor's degree",
 "Title: Enrollment Specialist Team Lead\nSkills: Customer service, Goal achievement, Teamwork, Communication, Retail knowledge, Sales optimization, Product knowledge, Competitive environment, Team meetings, Hourly wage, Medical coverage, Dental coverage, Vision coverage, Paid time off, Advancement opportunities, Physical ability, Driver's license, Lifting",
 'Title: Senior Risk Manager, Interest Rate Risk in the Banking Book - Hybrid\nSkills: Interest Rate Risk Management, Asset Liability Management, Balance Sheet Management, Treasury/Finance, Risk Management, Risk Quantification, Risk Governance, Risk Controls, Forecasting, Risk Reporting, Risk Identification, Risk Policies, Risk Limits, Risk Appetite, Stres

In [48]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types


# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})


class GeminiEmbeddingFunction(EmbeddingFunction):
    # def __init__(self, document_mode=True):
    #     self.document_mode = document_mode
    #     self.client = genai

    # @retry.Retry(predicate=is_retriable)
    # def __call__(self, input):
    #     if self.document_mode:
    #         task_type = "retrieval_document"
    #     else:
    #         task_type = "retrieval_query"

    #     # Ensure input is a list of strings (Chroma passes it this way)
    #     response = self.client.embed_content(
    #         model="models/text-embedding-004",
    #         content=input,
    #         task_type=task_type,
    #     )

    #     return [e.values for e in response.embeddings]
    
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

In [50]:
import chromadb

DB_NAME = "skillsdb"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

In [51]:
db.count()
db.peek(1)

{'ids': ['0'],
 'embeddings': array([[ 4.60675173e-03,  4.76415968e-04, -6.02308139e-02,
         -1.63157433e-02,  4.80655953e-02,  1.70870135e-05,
          2.55073216e-02,  1.13362912e-02, -7.99719896e-03,
          5.94347790e-02,  8.94606952e-03,  2.68731499e-03,
          8.55240896e-02, -1.04237664e-02, -4.03682813e-02,
         -1.39879491e-02,  2.51365602e-02,  1.40086776e-02,
         -3.23391519e-02, -4.80190217e-02,  3.04118730e-02,
         -5.12191616e-02,  8.73220060e-03, -4.27141599e-02,
         -3.18831988e-02,  2.23306827e-02,  1.39725273e-02,
         -4.29865383e-02, -7.76849464e-02, -7.99402222e-03,
          4.68154177e-02,  3.20878066e-02, -3.02996896e-02,
         -4.96860780e-02, -2.11113933e-02,  1.10167833e-02,
         -2.18921956e-02,  7.63716688e-03,  4.44993675e-02,
         -3.36979665e-02, -2.39753984e-02, -1.38910217e-02,
         -2.32827328e-02,  1.61139611e-02, -2.74079051e-02,
         -8.01222678e-03, -2.78923828e-02,  2.56166160e-02,
         -5

In [52]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "What skills are needed for a Data Engineer?"

result = db.query(query_texts=[query], n_results=1)
[all_passages] = result["documents"]

Markdown(all_passages[0])

Title: Software Engineer
Skills: SQL, Python, Django, HTML, CSS, JavaScript, Bootstrap, Microsoft Visual C# .net, Microsoft Visual Basic .net, VBScript, ETL, SSIS, DTS, Azure DevOps, Production Support, Healthcare, Teamwork, Collaboration, Respect, Member Centric, Honesty, Integrity, Commitment to Service Excellence, Compassion, Empathy, Problem Solving, Initiative, SelfDirected, Flexibility, Adaptability, Communication, Strategic Thinking, Leadership, Employee Development, Microsoft SQL Server 20002017, Pandas, XlsxWriter, Xlrd, Openpyxl, Pyodbc, SQLAlchemy, Internet Information Services (IIS)

In [53]:
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative bot that gives advice on skills needed for a specific job using text from the reference job posting included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
"""

# Add the retrieved documents to the prompt.
for passage in all_passages:
    passage_oneline = passage.replace("\n", " ")
    prompt += f"PASSAGE: {passage_oneline}\n"

print(prompt)

You are a helpful and informative bot that gives advice on skills needed for a specific job using text from the reference job posting included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: What skills are needed for a Data Engineer?
PASSAGE: Title: Software Engineer Skills: SQL, Python, Django, HTML, CSS, JavaScript, Bootstrap, Microsoft Visual C# .net, Microsoft Visual Basic .net, VBScript, ETL, SSIS, DTS, Azure DevOps, Production Support, Healthcare, Teamwork, Collaboration, Respect, Member Centric, Honesty, Integrity, Commitment to Service Excellence, Compassion, Empathy, Problem Solving, Initiative, SelfDirected, Flexibility, Adaptability, Communication, Strategic Thinking, Leadership, Employee Development, Microsoft SQL Server 20002017, Pandas, XlsxWriter, Xlrd, Openpyxl, Pyodbc, SQLAlchemy, Internet Information Services (IIS)



In [54]:
answer = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt)

Markdown(answer.text)

Based on the job posting, the skills needed for a Data Engineer include SQL, Python, ETL, SSIS, DTS, Azure DevOps, Pandas, XlsxWriter, Xlrd, Openpyxl, Pyodbc, and SQLAlchemy.




# Section 4: Evaluation

In [55]:
import enum

# Define the evaluation prompt
SUMMARY_PROMPT = """\
# Instruction
You are an expert evaluator. Your task is to evaluate the quality of the responses generated by two AI models. We will provide you with the user input and a pair of AI-generated responses (Response A and Response B).
You should first read the user input carefully for analyzing the task, and then evaluate the quality of the responses based on the Criteria provided in the Evaluation section below.
You will first judge responses individually, following the Rating Rubric and Evaluation Steps.
Then you will give step-by-step explanations for your judgment, compare results to declare the winner based on the Rating Rubric and Evaluation Steps.

# Evaluation
## Metric Definition
You will be assessing groundedness, which measures the ability to provide or reference information included only in the user prompt.

## Criteria
Groundedness: The response contains information included only in the user prompt. The response does not reference any outside information.

## Rating Rubric
"A": Response A is more grounded than Response B.
"SAME": Both response A and B are equally grounded, or ungrounded.
"B": Response B is more grounded than Response A.

## Evaluation Steps
STEP 1: Analyze Response A based on the groundedness criteria: Identify any information in the response not present in the prompt and provide assessment according to the criterion.
STEP 2: Analyze Response B based on the groundedness criteria: Identify any information in the response not present in the prompt and provide assessment according to the criterion.
STEP 3: Compare the overall performance of Response A and Response B based on your analyses and assessment.
STEP 4: Output your preference of "A", "SAME" or "B" to the pairwise_choice field according to the Rating Rubric.
STEP 5: Output your assessment reasoning in the explanation field.

# User Inputs and AI-generated Responses
## User Inputs
### Prompt
{prompt}

## AI-generated Responses

### Response A
{baseline_model_response}

### Response B
{response}
"""

# Define a structured enum class to capture the result.
class SummaryRating(enum.Enum):
  VERY_GOOD = '5'
  GOOD = '4'
  OK = '3'
  BAD = '2'
  VERY_BAD = '1'


def eval_summary(prompt, baseline_model_response, ai_response):
  """Evaluate the generated summary against the prompt used."""

  chat = client.chats.create(model='gemini-2.0-flash')

  # Generate the full text response.
  response = chat.send_message(
      message=SUMMARY_PROMPT.format(prompt=prompt, baseline_model_response= baseline_model_response, response=ai_response)
  )
  verbose_eval = response.text

  # Coerce into the desired structure.
  structured_output_config = types.GenerateContentConfig(
      response_mime_type="text/x.enum",
      response_schema=SummaryRating,
  )
  response = chat.send_message(
      message="Convert the final score.",
      config=structured_output_config,
  )
  structured_eval = response.parsed

  return verbose_eval, structured_eval


text_eval, struct_eval = eval_summary(prompt=prompt, baseline_model_response = baseline_response, ai_response = answer.text)
Markdown(text_eval)

STEP 1: Response A is completely irrelevant to the prompt. All information is not from the provided passage.
STEP 2: Response B provides relevant information from the passage.
STEP 3: Response B is more grounded than response A because response A does not contain any information from the passage.
STEP 4: B
STEP 5: Response A is completely irrelevant to the prompt. Response B is more grounded as all listed skills are directly from the provided job posting.